In [103]:
# Imports and such
import numpy as np
import pandas as pd
import os
from IPython.display import clear_output
np.set_printoptions(edgeitems=60, linewidth = 1000000, )
np.random.seed(1007)
cards = np.array(['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'])
suits = np.array(['s', 'o'])

In [104]:
# Creates complete range
def fullRange(cards):
    hands = np.empty([len(cards), len(cards)], dtype='<U3')
    for i in np.arange(len(cards)):
        for j in np.arange(len(cards)):
            if i == j:
                hands[i, j] = f"{cards[i]}{cards[j]}"
            elif i < j:
                hands[i, j] = f"{cards[i]}{cards[j]}s"
            else:
                hands[i, j] = f"{cards[j]}{cards[i]}o"
    
    return hands

# Function to create a labeled poker range
def createRange(hands):
    output_range = np.empty_like(hands, dtype=object)
    rest_empty = False

    # Input for labeling the range
    position = input('Input position (UTG, MP, LJ, HJ, CO, BU, SB, BB): ')
    if position == '': return
    action = input('Input action (Call, OpenRaise, Raise) (blank is OpenRaise): ').lower()
    if action == '': action = "openraise"

    # If there is an opponent, determine their position and previous action
    if action != 'openraise': 
        opp = input('Is there an opponent? (y/n): ')
    
        if opp.lower() == 'y':
            opponent = input('Opponent position (UTG, MP, LJ, HJ, CO, BU, SB, BB): ')
            opp_action = input('Input opponent action: ')
            label = (f"{position}{action}_{opponent}{opp_action}_range.npy").lower()
    else:
        label = (f"{position}{action}_range.npy").lower()

    # Iterate over hands matrix
    for i in np.arange(len(hands)):  # Correct use of range
        row_empty = False
        for j in np.arange(len(hands[i])):  # Correct use of square brackets
            if row_empty or rest_empty:
                output_range[i, j] = '-'
                continue

            included = input(hands[i, j] + ' y/n/x/r: ')
            
            if included.lower() == 'x':
                output_range[i, j] = '-'
                rest_empty = True  # Skip remaining hands
            elif included.lower() == 'r':
                output_range[i, j] = '-'
                row_empty = True  # Skip remaining hands in this row
            elif included.lower() == 'y':
                output_range[i, j] = hands[i, j]
            else:
                output_range[i, j] = '-'

    # Save the range to a file
    np.save(label, output_range)
    print(f"{label} range saved as '{label}'")
    
    return output_range, label

# Gets range to quiz on
def get_range(path="."):
    output_range = None

    position = input('Which position (UTG, MP, LJ, HJ, CO, BU, SB, BB)? ').lower()
    if position == '': return
    action = input('Input action (Call, OpenRaise, Raise) (blank is OpenRaise): ').lower()
    if action == '': action = "openraise"
    opponent = ''
    opp_action = ''
    
    if action != 'openraise':
        opp = input('Is there an opponent? (y/n): ')
        
        if opp.lower() == 'y':
            opponent = input('Opponent position (UTG, MP, LJ, HJ, CO, BU, SB, BB): ').lower()
            opp_action = input('Input opponent action: ').lower()
            label = f"{position}{action}_{opponent}{opp_action}_range.npy"
    else:
        label = f"{position}{action}_range.npy"

    # Generate entry
    report = f"Position: {position.upper()}, Action: {action}, Opponent: {opponent.upper()}, Opponent Action: {opp_action.upper()}"

    for entry in os.listdir(path):
        if entry == label:
            output_range = np.load(label, allow_pickle=True)
            return output_range, label, report

    print(f"Couldn't find {label}. If it doesn't exist, please create it using the cells above")
    return

# Shuffles ranges for random quizzing
def shuffle_range(arr):
    flat_arr = arr.flatten()
    np.random.shuffle(flat_arr)
    shuffled_arr = flat_arr.reshape(arr.shape)
    return shuffled_arr

# Calculates the amount of hands marked correct compared to hands that should've been correct
def calculate_accuracy(player_range, correct_range):
    assert player_range.size == correct_range.size
    correct_count = np.sum(player_range == correct_range)
    total_hands = correct_range.size
    accuracy = correct_count / total_hands * 100
    return round(accuracy, 3)

# Creates a range of all the hands that were marked incorrect and should have been correct
def locate_missed_hands(player_range, correct_range):
    assert player_range.size == correct_range.size
    missed_hands = np.zeros_like(player_range)
    missed_hands.fill('-')
    for i in np.arange(len(player_range)):
        for j in np.arange(len(player_range[i])):
            if player_range[i, j] == '-' and correct_range[i, j] != '-':
                missed_hands[i, j] = correct_range[i, j]
            else:
                missed_hands[i, j] = '-'

    return missed_hands

# Generates the final table styled and such
def generate_result(player_range, correct_range, missed_hands, hands):
    result = np.empty_like(player_range)
    result.fill('-')
    
    # need to combine player_range and missed_hands
    for i in range(len(player_range)):
        for j in range(len(player_range[i])):
            if player_range[i, j] != '-':
                result[i, j] = player_range[i, j]
            elif missed_hands[i, j] != '-':
                result[i, j] = missed_hands[i, j]
            else:
                result[i, j] = '-'
    
    result_df = print_range(result)
    data = np.array(
        [
            # cell is not marked or supposed to be
            "gray"
            if cell == '-'
            else "green"
            # cell is in the correct range and was marked
            if any(cell == inner_cell for x in correct_range for inner_cell in x)
            and any(cell == inner_cell for x in player_range for inner_cell in x)
            and cell != '-'
            else "cyan"
            # cell is not in player range but in correct range
            if any(cell != inner_cell for x in player_range for inner_cell in x)
            and any(cell == inner_cell for x in correct_range for inner_cell in x)
            else "red"
            # cell is marked but shouldn't be
            if any(cell == inner_cell for x in player_range for inner_cell in x)
            and any(cell != inner_cell for x in correct_range for inner_cell in x)
            else "gray"
            for row in result_df.values
            for cell in row
        ]
    )

    cell_color = pd.DataFrame(data.reshape(result_df.shape), index=result_df.index, columns=result_df.columns)
    result_styled = result_df.style.set_table_styles([
                            {"selector": ".green", "props": "background-color:green; font-size:0.8rem;"},
                            {"selector": ".red", "props": "background-color:tomato; font-size:0.8rem;"},
                            {"selector": ".gray", "props": "background-color:gray; font-size:0.8rem;"},
                            {"selector": ".cyan", "props": "background-color:turquoise; font-size:0.8rem; color:black;"},
                        ])

    result_styled.set_td_classes(cell_color)
    return result_styled

# Fetches a desired poker range and quizzes the user on which hands are included in it
def quiz_range(hands):
    name = input('What is your name?')
    correct_range, _, entry = get_range()
    quiz_hands = hands
    # RANDOM NOT FINISHED YET
    # random_input = input('Quiz random hands? y/n').lower()
    random = False
    # if random_input == 'y': 
    #     random = True

    player_range = np.zeros_like(quiz_hands, dtype=object)
    player_range.fill('-')

    # Shuffle ranges
    if random: 
        correct_range = shuffle_range(correct_range)
        quiz_hands = shuffle_range(quiz_hands)

    # Loops through all possible hands, asking if each hand is in the correct range or not
    # Options are: y = yes, n = no, r = no more in row, x = no more in range
    rest_empty = False
    for i in np.arange(len(quiz_hands)):
        row_empty = False
        for j in np.arange(len(quiz_hands[i])):
            accuracy = calculate_accuracy(player_range, correct_range)
            if accuracy == 100:
                print(player_range, "\nFound all possible hands\nAccuracy: ", accuracy)

                # Save score
                score = f"{name} {accuracy}: {entry}"
                with open("scores.csv", 'a') as file:
                    file.write(score)

                return
            if rest_empty:
                break
            if row_empty:
                continue
            
            clear_output(wait=True)
            display(print_range(player_range))
            answer = input("Is " + quiz_hands[i, j] + " in this range? y/n/r/x")
            if answer.lower() == 'y':
                player_range[i, j] = quiz_hands[i, j]
            elif answer.lower() == 'r':
                row_empty = True
            elif answer.lower() == 'x':
                rest_empty = True
        row_empty = False
    
    accuracy = calculate_accuracy(player_range, correct_range)
    missed_hands = locate_missed_hands(player_range, correct_range)

    # Create final df that has correct cells in green, missing in red, and rest in grey
    clear_output(wait=True)
    df = generate_result(player_range, correct_range, missed_hands, hands)
    display(df)
    print("Accuracy: ", accuracy, "%")

    # Save score
    score = f"{name} {accuracy}: {entry}"
    with open("scores.csv", 'a') as file:
        file.write(score)
        file.write("\n")

    return 
    
# Pretty displays the range
def print_range(r):

    column_labels = cards
    row_labels = cards

    df = pd.DataFrame(r, columns=column_labels, index=row_labels)
    return df

# Supply a full file name and turn all blanks/spaces into '-'
def remove_blanks(label):
    r = np.load(f"{label}", allow_pickle=True)
    for i in range(len(r)):
        for j in range(len(r[i])):
            if r[i, j] == '':
                r[i, j] = '-'
    np.save(f'{label}', r)

In [105]:
### RUN ME ###
hands = fullRange(cards)

create = input("Do you want to create/edit a new range? y/n")
while create.lower() == 'y':
    # CREATE A RANGE
    edit = input("Do you want to edit a range? y/n")
    if edit == 'y':
        output_range, label, _ = get_range()
        display(print_range(output_range))
        hand = input('Range is displayed at bottom of page. Which hand do you want to change? ex: AKs')
        for i in np.arange(len(output_range)):
            for j in np.arange(len(output_range)):
                if hands[i, j].lower() == hand.lower():
                    o = input("Do you want to add or remove this hand from the range? ('add'/'remove')")
                    if o.lower() == 'add':
                        output_range[i, j] = hands[i, j]
                    elif o.lower() == 'remove':
                        output_range[i, j] = '-'
                    display(print_range(output_range))
                    # Save the range to a file
                    np.save(label, output_range)
    else:
        output_range, label = createRange(hands)
        display(print_range(output_range))
    create = input("Do you want to create/edit another range? y/n")

view = input("Do you want to view a range? y/n")
while view == 'y':
    # VIEW A RANGE
    output_range,_, _ = get_range()
    display(print_range(output_range))
    view = input("Do you want to view another range? y/n")

quiz = input("Do you want to quiz a range? y/n")
while quiz.lower() == 'y':
    # QUIZ A RANGE
    quiz_range(hands)
    quiz = input("Finished quiz, check output at bottom of page. Do you want to quiz another range? y/n")

,A,K,Q,J,T,9,8,7,6,5,4,3,2
A,AA,AKs,AQs,AJs,ATs,-,A8s,-,-,A5s,A4s,A3s,-
K,AKo,KK,KQs,KJs,KTs,-,-,-,-,-,-,-,-
Q,AQo,-,QQ,-,-,-,-,-,-,-,-,-,-
J,-,-,-,JJ,-,-,-,-,-,-,-,-,-
T,-,-,-,-,TT,-,-,-,-,-,-,-,-
9,-,-,-,-,-,99,-,-,-,-,-,-,-
8,-,-,-,-,-,-,88,87s,-,-,-,-,-
7,-,-,-,-,-,-,-,77,76s,-,-,-,-
6,-,-,-,-,-,-,-,-,66,65s,-,-,-
5,-,-,-,-,-,-,-,-,-,-,-,-,-
